# Week 8 Further Readings

We'll explore some useful packages and modules in the Biopython library. Even though we will not be covering Biopython in the course, it is helpful to have a broad idea of what it offers so that you can use it in your research or whenever required. The idea behind creating this notebook is to introduce you to Biopython so that it serves as a starter for you to explore the vast capabilities of Biopython.

Before we dive into Biopython, I'd like to take a moment to mention Python 3.13 was just released last week! Some notable updates include improvements to the Python interactive interpreter and error messages. You can find all the improvements and new features of Python 3.13 in the [What's new in Python 3.13](https://docs.python.org/3/whatsnew/3.13.html) page.


## Biopython

Biopython is a collection of Python modules for computational molecular biology. [Biopython website](http://www.biopython.org) provides an online resource for modules, scripts, and web links for developers of Python-based software for bioinformatics use and research. It includes parsers for various bioinformatics file formats like BLAST, FASTA, Genbank, etc., access to online databases like NCBI, PDB etc., a standard sequence class, sequence alignment, and many more.

To get started with Biopython, we first need to install it as it is not a part of the standard library which simply means that it does not come pre-installed with Python like other packages such as `sys`, `os`, `argparse` etc. You can either install it using `pip` or directly into a mamba environment. Make sure to also install `jupyter lab` in the same environment to work through the examples in the notebook.

In [ ]:
pip install biopython

Or

In [ ]:
mamba env -n biopython -c conda-forge biopython

For installation instructions, refer to the [README file.](https://github.com/biopython/biopython/blob/master/README.rst)

Once installed, we can check the Biopython version installed using the `__version__` dunder variable.

In [1]:
import Bio
print(Bio.__version__)

1.84


### Bio.Seq Module

One of the most common data that Bioinformaticians deal with is the biological sequence data which can essentially be thought of as strings and the sequence processing as some sort of string manipulation. Biopython's `Seq` module provides `Seq` class to deal with biological sequences.

In [2]:
seq = "ATGACTAGACTACATAGCATGACTACGAATCATAAATAG"

In [3]:
from Bio.Seq import Seq

bpy_seq = Seq(seq)

#check the type of the bpy_seq object
print(type(bpy_seq))

<class 'Bio.Seq.Seq'>


Similar to Python `string`, Biopython `Seq` object is also immutable. We see an error if we try to change the last three bases.

In [4]:
bpy_seq[-3:] = "TAA"

TypeError: 'Seq' object does not support item assignment

However, Biopython also provides mutable version of `Seq` object via the `MutableSeq` class. 

In [5]:
from Bio.Seq import MutableSeq
mut_seq = MutableSeq(seq)
print(mut_seq)
print(type(mut_seq))

ATGACTAGACTACATAGCATGACTACGAATCATAAATAG
<class 'Bio.Seq.MutableSeq'>


Now, we can perform modifications to the `mut_seq` variable in place without getting an error.

In [6]:
mut_seq[-3:] = "TAA"
print(mut_seq)

ATGACTAGACTACATAGCATGACTACGAATCATAAATAA


One important thing to bear in mind is that when we make our sequence mutable, we cannot use it in places where an immutable object is required. A good example of this is dictionary keys which require that it is an immutable object. To work around this is simple, you can convert the `MutableSeq` into `Seq` object when you want to make it immutable again.

In [7]:
immut_seq = Seq(mut_seq)
print(immut_seq)
print(type(immut_seq))

ATGACTAGACTACATAGCATGACTACGAATCATAAATAA
<class 'Bio.Seq.Seq'>


<br>

`Seq` object supports many of the methods that a Python `string` object does, but note that some methods behave differently even though they are named the same(eg, `translate()`) and there are also additional biologically relevant methods like `reverse_complement()`. Let us now explore some `Seq` object methods. 

Similar to `str` method `count()`, `Seq` object's method `count()` also returns a non-overlapping count. However, if you are looking to count the number of occurrences of an overlapping pattern, you can use `count_overlap()`.

In [8]:
seq = Seq("ATGACTAGACTACATAGAGACATGACTAGAGATCATAAATAG")

seq.count("AGA")

3

In [9]:
seq.count_overlap("AGA")

5

In the above sequence, without counting overlaps, `count()` only finds ATGACT**AGA**CTACAT**AGA**GACATGACT**AGA**GATCATAAATAG and returns 3. With `count_overlap()`, it finds the overlapping `AGA`s as well ATGACT**AGA**CTACAT**AGAGA**CATGACT**AGAGA**TCATAAATAG and returns 5.

Additionally, there are methods to complement, reverse-complement, transcribe, reverse-transcribe, and translate sequences.

In [10]:
print(f"Sequence:           {seq}")
#complement DNA sequence
comp_seq = seq.complement()
print(f"Complement:         {comp_seq}")

#reverse complement DNA sequence
rev_comp_seq = seq.reverse_complement()
print(f"Reverse complement: {rev_comp_seq}")

#transcribe DNA to RNA
rna_seq = seq.transcribe()
print(f"RNA sequence:       {rna_seq}")

#reverse-transcribe RNA to DNA
dna_seq = rna_seq.back_transcribe()
print(f"DNA sequence:       {dna_seq}")

#translate mRNA to protein
protein_seq = rna_seq.translate()
print(f"Protein sequence:   {protein_seq}")

Sequence:           ATGACTAGACTACATAGAGACATGACTAGAGATCATAAATAG
Complement:         TACTGATCTGATGTATCTCTGTACTGATCTCTAGTATTTATC
Reverse complement: CTATTTATGATCTCTAGTCATGTCTCTATGTAGTCTAGTCAT
RNA sequence:       AUGACUAGACUACAUAGAGACAUGACUAGAGAUCAUAAAUAG
DNA sequence:       ATGACTAGACTACATAGAGACATGACTAGAGATCATAAATAG
Protein sequence:   MTRLHRDMTRDHK*


Apart from these, methods like `startswith()`, `split()`, `index()`, `find()`, `join()` and so on work similarly to their `str` counterparts. You can find all the methods supported in the [documentation](https://biopython.org/docs/1.75/api/Bio.Seq.html).

You can find more `Seq` object manipulation examples [here](https://biopython.org/docs/latest/Tutorial/chapter_seq_objects.html).

### SeqIO Package

`SeqIO` package provides utilities to work with different biological file formats like `fasta`, `genbank`, multiple alignment files, etc. The main function we need to know about in this package is `SeqIO.parse()`. As the name suggests, it reads in or parses the sequence data and stores it as `SeqRecord` objects.

Let's look at how to use the `SeqIO.parse()` function. To illustrate this, we will use the Vibrio *cholerae* assembly file that we have been working with. The function takes two inputs, the filename and the file format. It returns an iterator that gives `SeqRecord` objects. If you remember other iterators we learned in class like `enumerate()`, `range()`, `zip()` and so on, the output produced by `SeqIO.parse()` function is similar and we can read it using a for loop.

In [11]:
from Bio import SeqIO

for record in SeqIO.parse("Vibrio_cholerae_N16961.fna", "fasta"):
    print(record)
    print("\n")

ID: NZ_CP028827.1
Name: NZ_CP028827.1
Description: NZ_CP028827.1 Vibrio cholerae strain N16961 chromosome 1, complete sequence
Number of features: 0
Seq('GTGTCATCTTCGCTATGGTTGCAATGTTTGCAACGGCTTCAGGAAGAGCTACCT...ATC')


ID: NZ_CP028828.1
Name: NZ_CP028828.1
Description: NZ_CP028828.1 Vibrio cholerae strain N16961 chromosome 2, complete sequence
Number of features: 0
Seq('TCAGTTTAGCTTACGCTGCACAAAATCAAGAATATCTTGCATCAGCTCATCATC...GAC')




In the above output, each sequence is stored as a `SeqRecord` object, which we can confirm using the `type()` function.

In [12]:
print(type(record))

<class 'Bio.SeqRecord.SeqRecord'>


We can also peek into the contents of the `record` object using `repr()`.

In [13]:
print(repr(record))

SeqRecord(seq=Seq('TCAGTTTAGCTTACGCTGCACAAAATCAAGAATATCTTGCATCAGCTCATCATC...GAC'), id='NZ_CP028828.1', name='NZ_CP028828.1', description='NZ_CP028828.1 Vibrio cholerae strain N16961 chromosome 2, complete sequence', dbxrefs=[])


As we can see, the `SeqIO.parse()` function automatically parses the fasta file, creates an object for each record and stores the sequence ID, and other metadata in the header line as description, and the sequence. This is more convenient than reading the file as a generic text file and having to parse through the file line by line to handle the header and sequence lines separately. Interestingly, we can see the 'Number of features' is 0 for both sequences. Let us see why that is in a minute.

You can easily access the record's attributes like `ID`, `Description` etc., using the `.`(dot) notation. 

In [14]:
print(record.id)
print(record.description)
print(len(record.seq)) #number of bases in the sequence

NZ_CP028828.1
NZ_CP028828.1 Vibrio cholerae strain N16961 chromosome 2, complete sequence
1072331


A sensible data type to store the sequences and their attributes is a dictionary with the sequence ID as the key. `SeqIO` also has `.to_dict()` method to directly convert the `SeqRecord` into a `dict`.

In [15]:
seq_dict = SeqIO.to_dict(SeqIO.parse("Vibrio_cholerae_N16961.fna", "fasta"))

#check the type
print(type(seq_dict))
for seq in seq_dict.items():
    print(seq)
    print("\n")

<class 'dict'>
('NZ_CP028827.1', SeqRecord(seq=Seq('GTGTCATCTTCGCTATGGTTGCAATGTTTGCAACGGCTTCAGGAAGAGCTACCT...ATC'), id='NZ_CP028827.1', name='NZ_CP028827.1', description='NZ_CP028827.1 Vibrio cholerae strain N16961 chromosome 1, complete sequence', dbxrefs=[]))


('NZ_CP028828.1', SeqRecord(seq=Seq('TCAGTTTAGCTTACGCTGCACAAAATCAAGAATATCTTGCATCAGCTCATCATC...GAC'), id='NZ_CP028828.1', name='NZ_CP028828.1', description='NZ_CP028828.1 Vibrio cholerae strain N16961 chromosome 2, complete sequence', dbxrefs=[]))




The content of the `dict` looks the same as the output of `repr()` we saw above. To access the element, we can use the sequence ID as the key.

In [16]:
print(seq_dict['NZ_CP028827.1'].id)
print(seq_dict['NZ_CP028827.1'].seq[:1000]) #print first 1000 bases

NZ_CP028827.1
GTGTCATCTTCGCTATGGTTGCAATGTTTGCAACGGCTTCAGGAAGAGCTACCTGCCGCAGAATTCAGTATGTGGGTGCGTCCGCTTCAAGCGGAGCTCAATGACAATACTCTCACTTTATTCGCCCCGAACCGCTTTGTGTTGGATTGGGTACGCGATAAGTACCTCAATAACATCAATCGTCTGCTGATGGAATTCAGTGGCAATGATGTGCCTAATTTGCGCTTTGAAGTGGGGAGCCGCCCTGTGGTGGCGCCAAAACCCGCGCCTGTACGTACGGCTGCGGATGTCGCGGCGGAATCGTCGGCGCCTGCGCAATTGGCGCAGCGTAAACCTATCCATAAAACCTGGGATGATGACAGTGCTGCGGCTGATATTACTCACCGCTCAAATGTGAACCCGAAACACAAGTTCAACAACTTCGTGGAAGGTAAATCTAACCAGTTAGGTCTGGCCGCGGCTCGCCAAGTCTCTGATAACCCAGGTGCGGCGTATAACCCCCTCTTTTTGTATGGCGGCACCGGTTTGGGTAAAACGCACTTGCTGCATGCGGTGGGTAACGCGATTGTTGATAACAACCCGAACGCTAAAGTGGTGTACATGCACTCTGAGCGTTTCGTGCAAGACATGGTAAAAGCCCTGCAGAACAACGCGATTGAAGAATTCAAACGCTACTATCGCAGTGTAGATGCCTTGTTGATCGACGATATTCAATTCTTTGCCAACAAAGAGCGTTCGCAGGAAGAGTTCTTCCACACCTTTAACGCACTGCTGGAAGGCAACCAACAAATTATTTTGACTTCTGACCGCTATCCAAAAGAGATCAGTGGTGTAGAAGATCGTCTCAAATCGCGTTTTGGCTGGGGCTTAACGGTGGCGATCGAGCCGCCGGAGTTGGAAACCCGCGTCGCGATCTTGATGAAAAAAGCGGAAGATCACCAGATTCATCTGCCGGATGAAGTGGCTTTCTTTATTGCGAAACGCCT

NOTE: Using the `to_dict()` function will load all the records into memory at once, and therefore is not recommended when working with very large files. Instead, `Bio.SeqIO` package provides dictionary-like access to any record through the `index()` function. The only drawback with this function is that it supports a limited number of input file formats like fasta, fastq but not alignment file types like phylip.

Now let us understand why the "number of features" is zero when we read in the fasta file and what we can do about it to store the feature information. When we read data from a fasta file, the `SeqIO.parse()` function has no idea what features are present in the sequence and treats it as a plain sequence. We need to annotate the sequence with the coordinates of the features using a feature-encoding file like BED file or GFF file and store it in the `SeqRecord` object.

All feature-related information can be stored using the `SeqFeature` object. The key idea of the `SeqFeature` object is to describe a region in a sequence. As a result, the main attributes supported include `type` (type of feature like gene, CDS, etc), `location`(coordinates of the feature) and `qualifier` (a dictionary to store any metadata about the feature). The `location` attribute is encoded using the `SimpleLocation` object which stores start, stop and strand information. You can find more details about `SeqFeature` and `SimpleLocation` objects [here](https://biopython.org/docs/latest/Tutorial/chapter_seq_annot.html#feature-location-and-position-objects).

Let us look at a demo to store the features in the "Vibrio_cholerae_N16961.bed" BED file and annotate the sequence with the features. We'll use it to then extract the **glnL** gene sequence which we are quite familiar with.

In [17]:
from Bio.Seq import Seq
from Bio.SeqFeature import SeqFeature, SimpleLocation

#read in the fasta file
records = SeqIO.parse("Vibrio_cholerae_N16961.fna", "fasta")

#list to store the SeqFeature objects for every feature in bed file
features = []

#read in the contents of the bed file
with open("Vibrio_cholerae_N16961.bed") as bed_file:
    for line in bed_file:
        feature = line.strip().split("\t")
        bed_id = feature[0]
        start = int(feature[1])
        stop = int(feature[2])
        gene = feature[3]
        strand = 1 if feature[5] == "+" else -1 #strand is encoded as 1 for forward strand and -1 for reverse strand
        location = SimpleLocation(start, stop, strand=strand) #object to store the coordinates and the strand information
        feature = SeqFeature(location, type="gene", qualifiers={"gene": gene, "bed_id": bed_id}) #object to store the feature information
        features.append(feature) 

#annotate the sequence with feature object created above
for record in records:
    for feature in features:
        if record.id == feature.qualifiers["bed_id"]: #match the sequence IDs
            record.features.append(feature)
    print(record)
    print("\n")

    #extarct glnL gene sequence 
    for feature in record.features:
        if feature.qualifiers["gene"] == "glnL":
            glnl_seq = feature.extract(record.seq)
            print(">glnL")
            print(glnl_seq)
            print("\n")

ID: NZ_CP028827.1
Name: NZ_CP028827.1
Description: NZ_CP028827.1 Vibrio cholerae strain N16961 chromosome 1, complete sequence
Number of features: 2583
Seq('GTGTCATCTTCGCTATGGTTGCAATGTTTGCAACGGCTTCAGGAAGAGCTACCT...ATC')


>glnL
GTGAGTGCAGAATTAAGCCAAACCATCATTAATAATCAGGTCACATCAGTGCTCATTTTGGACGAGTCACTGATGATTCGCTACGCCAACCCTGCCGCTGAACAGCTGTTTTCACAAAGTGCTAAACGCTTGATGCATCAAAGCTTAAATCATTTAGTGCAACACTCCTCTCTCGATTTACAACTGCTCACGCAGCCACTCCAGAGCGGACAAAGCATTACTGACAGCGATGTCACCTTGGTGATCGATAGCAAACCCTTAATGCTCGAAGTCACCGTCAGCCCGATTTCTTGGCACAAAGAGCTGCTGTTACTGGCCGAAATGCGCACGATTGGTCAACAACGCCGGCTAACCCAAGAACTCAATCAACACGCTCAGCAACAAGCGGCTAAGTTATTGGTCAGAGGCTTGGCTCATGAAATCAAAAATCCTTTGGGTGGTTTAAGAGGTGCGGCCCAGCTCTTAGAGCGTATGCTTCCCGATCCGGCCCTGATGGAATATACCCAAATCATCATCGAACAGGCAGATCGCTTGCGGGGATTGGTTGATCGCTTACTCGGCCCGCAACGTCCGGGGGAGAAAAAATGGGAAAACCTTCACCTGATTTTGGAGAAGGTGCGTCAGTTGGTCGAGCTAGAAGCGGGCGCGAATTTGGTCTTTGAGCGCGATTATGACCCAAGTCTGCCGAATATTTTGATGGACACTGATCAAATCGAACAAGCCTTACTGAACATTGTCAGTAATGCGGCGCAAATTTTGACTAACCAAACGC

Notice above that the 'Number of features' for each sequence has been updated. Further, since the `SeqFeature` object for **glnL** has the information that it is encoded on the negative strand, reverse complementing it is automatically taken care of without any extra code.

We can take a look at the contents of the `SeqFeature` object by printing it.

In [18]:
print(record.features[0])

type: gene
location: [3:973](-)
qualifiers:
    Key: bed_id, Value: NZ_CP028828.1
    Key: gene, Value: N16961_RS13990



You can find more information on the capabilities of the `SeqIO` package in the documentation [here](https://biopython.org/docs/1.75/api/Bio.SeqIO.html). For more examples on handling `SeqRecord`, refer to the [tutorial](https://biopython.org/docs/latest/Tutorial/chapter_seq_annot.html).

### Phylo package

Biopython includes the `Phylo` package to work with phylogenetic trees. A phylogenetic tree depicts the evolutionary history of a group of species, organisms, or genes based on similarities and differences in their physical or genetic characteristics. If you remember from assignment 3, a common way to represent a phylogenetic tree or any dendrogram is Newick format. Using this package, we can perform tree-related operations like reading, drawing, traversing, coloring the branches, searching for nodes, rooting and unrooting, etc. Before we look at some examples, let us try to understand a few terminologies associated with phylogenetic trees to better interpret the outputs we see.

* Root - the most recent common ancestor of all the species in the tree
* Branch - a line that connects nodes in the tree representing evolutionary relationships and the passage of genetic changes.
* Branch length - indicates the amount of evolutionary change or time.
* Clade - group of nodes that consists of a single common ancestor and all its descendants.
* Confidence/Bootstrap values - a statistical measure of confidence that can be placed in a specific branch or clade, higher values indicate higher confidence.
* Rooted tree - a tree with a single root represents the most recent common ancestor of all the entities in the tree and shows the direction of evolutionary time.
* Unrooted tree - a tree that does not have a root, thus doesn't indicate the direction of evolution but represents the relationships among species.

We'll look at a few phylogenetic tree operations here using the tree.nwk file.

In [19]:
!cat tree.nwk

(Gallus_gallus:0.04596,((Homo_sapiens:0.00957,Pan_troglodytes:0.03864)100:0.01549,
(Rattus_norvegicus:0.03107,Mus_musculus:0.01651)91:0.00398)99:0.01629)

In [20]:
from Bio import Phylo
tree = Phylo.read("tree.nwk", "newick")
print(tree)

Tree(rooted=False, weight=1.0)
    Clade()
        Clade(branch_length=0.04596, name='Gallus_gallus')
        Clade(branch_length=0.01629, confidence=99)
            Clade(branch_length=0.01549, confidence=100)
                Clade(branch_length=0.00957, name='Homo_sapiens')
                Clade(branch_length=0.03864, name='Pan_troglodytes')
            Clade(branch_length=0.00398, confidence=91)
                Clade(branch_length=0.03107, name='Rattus_norvegicus')
                Clade(branch_length=0.01651, name='Mus_musculus')


The `read()` function returns a `Tree` object that contains all the information about the tree, such as whether it’s rooted or unrooted. It has one root clade, and under that, it’s nested lists of clades down to the nodes.

`draw_ascii()` creates a simple ASCII-art (plain text) dendrogram to visualize the tree.

In [21]:
Phylo.draw_ascii(tree)

  _____________________________________ Gallus_gallus
 |
_|                          _______ Homo_sapiens
 |             ____________|
 |            |            |________________________________ Pan_troglodytes
 |____________|
              |   __________________________ Rattus_norvegicus
              |__|
                 |_____________ Mus_musculus



We can search for specific clades by their names using the `.find_names()` method. We can then use this information to perform other operations like finding the distance between them.

In [22]:
clade1 = tree.find_any(name="Homo_sapiens")
clade2 = tree.find_any(name="Pan_troglodytes")

distance = tree.distance(clade1, clade2)
print(f"Distance between {clade1} and {clade2} is {distance}")

Distance between Homo_sapiens and Pan_troglodytes is 0.04821


There are additional operations you can perform with `Phylo` package its subpackages and modules. You can read more about them in the [documentation.](https://biopython.org/docs/latest/Tutorial/chapter_phylo.html) 

In summary, Biopython offers a wide range of powerful packages and modules beyond what we've looked at, including tools for running BLAST, performing pairwise and multiple sequence alignments, population genetics, motif analysis, clustering algorithms, and structural biology to name a few. Whatever type of biological data you're working with, chances are that Biopython has a package to help you process it. The best place to look for all that it offers is the [Biopython documentation](https://biopython.org/docs/latest/index.html) which is extensive with plenty of usage examples. They also have helpful [tutorials](https://biopython.org/wiki/Category:Wiki_Documentation) to follow along.